# pythreejs https://github.com/jovyan/pythreejs

## A Jupyter - threejs bridge

Pythreejs is a jupyter interactive widget bringing fast WebGL 3d visualization to the Jupyter notebook.

- Originally authored by Jason Grout
- BSD Licensed

Pythreejs is *not* a 3d plotting library, it only exposes the threejs scene objects to the Jupyter kernel.

**Installation:**

```bash
conda install -c conda-forge pythreejs
```

In [1]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text
from traitlets import link, dlink

## Simple Sphere and Animation

In [2]:
ball = Mesh(geometry=SphereGeometry(radius=1), 
            material=MeshLambertMaterial(color='red'),
            position=[2, 1, 0])

c = PerspectiveCamera(position=[0, 5, 5], up=[0, 1, 0],
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

scene = Scene(children=[ball, c, AmbientLight(color='#777777')])

renderer = Renderer(camera=c, 
                    scene=scene, 
                    controls=[OrbitControls(controlling=c)],
                   width=500, height=500)
display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 5.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), controls=[OrbitControls(controlling=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 5.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)))], scene=Scene(children=(Mesh(geometry=SphereGeometry(), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='red', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), position=(2.0, 1.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 5.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), AmbientLight(color='#777777', quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap())

In [3]:
ball.geometry.radius = 0.5

In [4]:
import time, math
ball.material.color = '#4400dd'
for i in range(1, 150, 2):
    ball.geometry.radius = i / 100.
    ball.position = [math.cos(i / 10.), math.sin(i / 50.), i / 100.]
    time.sleep(.05)

# Clickable Surface



In [5]:
# Generate surface data:
view_width = 600
view_height = 400
nx, ny = (20, 20)
xmax=1
x = np.linspace(-xmax, xmax, nx)
y = np.linspace(-xmax, xmax, ny)
xx, yy = np.meshgrid(x, y)
z = xx ** 2 - yy ** 2
#z[6,1] = float('nan')


# Generate scene objects from data:
surf_g = SurfaceGeometry(z=list(z[::-1].flat), 
                         width=2 * xmax,
                         height=2 * xmax,
                         width_segments=nx - 1,
                         height_segments=ny - 1)

surf = Mesh(geometry=surf_g,
            material=MeshLambertMaterial(map=height_texture(z[::-1], 'YlGnBu_r')))

surfgrid = SurfaceGrid(geometry=surf_g, material=LineBasicMaterial(color='black'),
                       position=[0, 0, 1e-2])  # Avoid overlap by lifting grid slightly

# Set up picking bojects:
hover_point = Mesh(geometry=SphereGeometry(radius=0.05),
                   material=MeshLambertMaterial(color='hotpink'))

click_picker = Picker(controlling=surf, event='dblclick')
hover_picker = Picker(controlling=surf, event='mousemove')

# Set up scene:
key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.4)
c = PerspectiveCamera(position=[0, 3, 3], up=[0, 0, 1], aspect=view_width / view_height,
                      children=[key_light])

scene = Scene(children=[surf, c, surfgrid, hover_point, AmbientLight(intensity=0.8)])

renderer = Renderer(camera=c, scene=scene,
                    width=view_width, height=view_height,
                    controls=[OrbitControls(controlling=c), click_picker, hover_picker])


# Set up picking responses:
# Add a new marker when double-clicking:
def f(change):
    value = change['new']
    print('Clicked on %s' % value)
    point = Mesh(geometry=SphereGeometry(radius=0.05), 
                 material=MeshLambertMaterial(color='red'),
                 position=value)
    scene.add(point)

click_picker.observe(f, names=['point'])

# Have marker follow picker point:
link((hover_point, 'position'), (hover_picker, 'point'))

# Show picker point coordinates as a label:
h = HTML()
def g(change):
    h.value = 'Green point at (%.3f, %.3f, %.3f)' % tuple(change['new'])
g({'new': hover_point.position})
hover_picker.observe(g, names=['point'])
display(h)
display(renderer)

HTML(value='Green point at (0.000, 0.000, 0.000)')

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.4, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 3.0, 3.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0)), controls=[OrbitControls(controlling=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.4, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(0.0, 3.0, 3.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0))), Picker(controlling=Mesh(geometry=SurfaceGeometry(attributes={'position': <BufferAttribute shape=(400, 3), dtype=float32>, 'index': <BufferAttribute shape=(2166,), dtype=uint16>, 'normal': <BufferAttribute shape=(400, 3), dtype=float32>, 'uv': <BufferAttribute shape=(400, 2), dtype=float32>}), material=MeshLambertMaterial(alphaMap=None, aoMap=None, emissiveMap=None, envMap=None, lightMap=None, map=DataTexture(data=array([[[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ..., 
        [ 32, 118, 179, 255],
        [ 36, 152, 193, 255],
        [ 66, 182, 196, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ..., 
        [ 40, 156, 193, 255],
        [ 66, 182, 196, 255],
        [115, 200, 189, 255]],

       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ..., 
        [ 66, 182, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       ..., 
       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ..., 
        [ 64, 181, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ..., 
        [ 40, 156, 193, 255],
        [ 64, 181, 196, 255],
        [115, 200, 189, 255]],

       [[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ..., 
        [ 32, 118, 179, 255],
        [ 36, 152, 193, 255],
        [ 66, 182, 196, 255]]], dtype=uint8), repeat=(1.0, 1.0)), specularMap=None), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), event='dblclick', object=None), Picker(controlling=Mesh(geometry=SurfaceGeometry(attributes={'position': <BufferAttribute shape=(400, 3), dtype=float32>, 'index': <BufferAttribute shape=(2166,), dtype=uint16>, 'normal': <BufferAttribute shape=(400, 3), dtype=float32>, 'uv': <BufferAttribute shape=(400, 2), dtype=float32>}), material=MeshLambertMaterial(alphaMap=None, aoMap=None, emissiveMap=None, envMap=None, lightMap=None, map=DataTexture(data=array([[[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ..., 
        [ 32, 118, 179, 255],
        [ 36, 152, 193, 255],
        [ 66, 182, 196, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ..., 
        [ 40, 156, 193, 255],
        [ 66, 182, 196, 255],
        [115, 200, 189, 255]],

       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ..., 
        [ 66, 182, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       ..., 
       [[165, 220, 183, 255],
        [109, 198, 190, 255],
        [ 66, 182, 196, 255],
        ..., 
        [ 64, 181, 196, 255],
        [109, 198, 190, 255],
        [165, 220, 183, 255]],

       [[115, 200, 189, 255],
        [ 66, 182, 196, 255],
        [ 40, 156, 193, 255],
        ..., 
        [ 40, 156, 193, 255],
        [ 64, 181, 196, 255],
        [115, 200, 189, 255]],

       [[ 66, 182, 196, 255],
        [ 36, 152, 193, 255],
        [ 32, 118, 179, 255],
        ..., 
        [ 32, 118, 179, 255],
        [ 36, 152, 193, 255],
        [ 66, 1

# Parametric Functions


To use the ParametricGeometry class, you need to specify a javascript function as a string. The function should take two parameters that vary between 0 and 1, and return a `new THREE.Vector3(x,y,z)`.

If you want to build the surface in Python, you'll need to explicitly construct the vertices and faces and build a basic geometry from the vertices and faces.

In [6]:
f = """
function f(origu,origv) {
    // scale u and v to the ranges I want: [0, 2*pi]
    var u = 2*Math.PI*origu;
    var v = 2*Math.PI*origv;
    
    var x = Math.sin(u);
    var y = Math.cos(v);
    var z = Math.cos(u+v);
    
    return new THREE.Vector3(x,y,z)
}
"""
surf_g = ParametricGeometry(func=f, slices=16, stacks=16);

surf = Mesh(geometry=surf_g, material=MeshLambertMaterial(color='green', side='FrontSide'))
surf2 = Mesh(geometry=surf_g, material=MeshLambertMaterial(color='yellow', side='BackSide'))
c = PerspectiveCamera(position=[5, 5, 3], up=[0, 0, 1],
                      children=[DirectionalLight(color='white',
                                                 position=[3, 5, 1],
                                                 intensity=0.6)])
scene = Scene(children=[surf, surf2, c, AmbientLight(intensity=0.5)])
renderer = Renderer(camera=c, scene=scene, controls=[OrbitControls(controlling=c)], width=400, height=400)
display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(5.0, 5.0, 3.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0)), controls=[OrbitControls(controlling=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(5.0, 5.0, 3.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0)))], scene=Scene(children=(Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='green', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='yellow', emissiveMap=None, envMap=None, lightMap=None, map=None, side='BackSide', specularMap=None), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)),), position=(5.0, 5.0, 3.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 0.0, 1.0)), AmbientLight(intensity=0.5, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap())